In [14]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

base_url = 'https://www.musicomh.com/classical/reviews-classical/page/'
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

reviews=[]

In [15]:
for page in range(1,107):
    print(page)
    url = base_url + str(page)
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")
    listings = soup.find_all('h2', attrs={"class": "title"}) + soup.find_all('div', attrs={"class": "image_text_box_index"})        

    for listing in listings:
        
        review = []        
        listing_url = listing.find('a').get('href')        
        r     = requests.get(listing_url, headers=headers)
        soup  = BeautifulSoup(r.text, "html5lib")
        
        try:
            posted = soup.find(attrs={"class": "date time published"}).get_text()
        except:
            posted = '31 Dec 2099'
        date   = datetime.strptime(posted, '%d %b %Y').date()
        
        try:
            info  = soup.find('div', attrs={"id": "post_title"}).find('h1').get_text()
            city  = info.split(',')[-1].strip()
            work  = info.split('@')[0].strip()
            if len(info.split('@')) > 1:
                venue = info.split('@')[1].split(',')[0].strip()
            else:
                venue = ''
        except:
            info = listing_url
            city = ''
            work = ''
            venue = ''

        try:
            rating = soup.find('div', attrs={"id": "post_title"}).parent.find('img')
            rating = rating.get('src').split('/')[-1].split('.')[0].replace('stars', '').replace('-', '.')
        except:
            rating = ''
            
        text  = soup.find('div', attrs={"class": "entry"}).find_all('p')
        full_text=''
        for w in text:
            if w.get('class')==None:
                full_text += w.get_text()
        
        review.append(info)
        review.append(work)
        review.append(venue)
        review.append(city)
        review.append(rating)        
        review.append(full_text)
        review.append(listing_url)
        review.append(date)        
        reviews.append(review)

import pandSource.('of_' + params[:content]).to_symas as pd
headers = ['title', 'work', 'venue', 'city', 'rating', 'full_text', 'url', 'date']
df = pd.DataFrame(reviews, columns=headers)
df.to_csv('../../data/processed/reviews/musicomh.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106


In [16]:
reviews[0]

['L’incoronazione di Poppea @ Longborough Festival Opera, Longborough',
 'L’incoronazione di Poppea',
 'Longborough Festival Opera',
 'Longborough',
 '4',
 'L’incoronazione di Poppea is a rather odd creation, given that it does not present the same tale of morality as lies at the heart of the vast majority of operas. With some exceptions such as Rigoletto, good usually triumphs, if only by remaining steadfast, so that even if the virtuous die the evil do not succeed. Monteverdi was, of course, writing before any operatic conventions with which we are now familiar were established. However, so conditioned are we today that, even knowing the story, it is hard to believe at the start that Ottone will not ultimately win Poppea while Nerone sees the error of his ways and returns to Ottavia.Instead, Nerone decides he wants Poppea and rides roughshod over a wise advisor (who he forces to commit suicide), the feelings of his wife and all sense of political responsibility in order to marry her.